# How To: Convert EMIT .nc to .envi

There are currently 2 similar methods to convert the EMIT netCDF4 files to `.envi` format. Note these only support L1B Radiance, L1B Obs, L2A Reflectance, L2A Reflectance Uncertainty, or L2A Mask to .envi. They do not yet support the L2B Mineral or L2B Mineral Uncertainty products.
1. The `write_envi` function in EMIT tools. This function is still being developed but will currently:
  - Write a GLT output to use for orthocorrection later 
  - Functions from `emit_tools` can be used beforehand to orthorectify if so desired
2. The `reformat.py` script available in the [emit-sds/emit-utils](https://github.com/emit-sds/emit-utils) repository can be used to convert EMIT netCDF files (as delivered to the LP DAAC) to ENVI files. This script also can apply the included GLT to orthorectify the image if desired.

This jupyter notebook walks through how to use both methods to provide users with programmatic routes to accomplish their EMIT reformatting workflows.

**Requirements:**
+ A NASA [Earthdata Login](https://urs.earthdata.nasa.gov/) account is required to download EMIT data   
+ Selected the `emit_tutorials` environment as the kernel for this notebook.
  + For instructions on setting up the environment, follow the the `setup_instructions.md` included in the `/setup/` folder of the repository.  

**Learning Objectives**
+ How to use the `write_envi` function from `emit_tools` module to convert an EMIT netCDF4 to a `.envi` file.
+ How to use the `reformat.py` function from the `emit-utils` repository to convert an EMIT netCDF4 to a `.envi` file.

## Setup

Import packages

In [1]:
import os
import earthaccess

### Authenticate using Earthdata Login and Download the required Granules

Login to your NASA Earthdata account and create a `.netrc` file using the `login` function from the `earthaccess` library. If you do not have an Earthdata Account, you can create one [here](https://urs.earthdata.nasa.gov/home). 

In [3]:
earthaccess.login(persist=True)

For this notebook we will download the files necessary using `earthaccess`. You can also access the data in place or stream it, but this can slow due to the file sizes. Provide a URL for an EMIT L2A Reflectance granule.

In [4]:
url = 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/EMITL2ARFL.001/EMIT_L2A_RFL_001_20220903T163129_2224611_012/EMIT_L2A_RFL_001_20220903T163129_2224611_012.nc'

Get an HTTPS Session using your earthdata login, set a local path to save the file, and download the granule asset - This may take a while, the reflectance file is approximately 1.8 GB. 

In [5]:
# Get Https Session using Earthdata Login Info
fs = earthaccess.get_fsspec_https_session()
# Retrieve granule asset ID from URL (to maintain existing naming convention)
granule_asset_id = url.split('/')[-1]
# Define Local Filepath
fp = f'../../data/emit/{granule_asset_id}'
# Download the Granule Asset if it doesn't exist
if not os.path.isfile(fp):
    fs.download(url, fp)

Now lets create an output folder where we will save the `.envi` files.

In [6]:
outpath = '../../data/emit/ag/envi' 
if not os.path.exists(outpath):
    os.makedirs(outpath)

## Method 1: Using `write_envi` from the `emit_tools` module.

Import the necessary packages for this method. 

In [7]:
import sys
sys.path.append('../../tools/emit/python/modules/')
import emit_tools as et

Open the granule using the `emit_xarray` function. We can orthorectify here if so desired.

In [8]:
import emit_tools as et

In [9]:
ds = et.emit_xarray(fp, ortho=True)
ds

<xarray.Dataset>
Dimensions:           (latitude: 2009, longitude: 2353, wavelengths: 285)
Coordinates:
  * wavelengths       (wavelengths) float32 381.0 388.4 ... 2.486e+03 2.493e+03
    fwhm              (wavelengths) float32 ...
    good_wavelengths  (wavelengths) float32 ...
  * latitude          (latitude) float64 -39.31 -39.31 -39.31 ... -40.4 -40.4
  * longitude         (longitude) float64 -62.51 -62.51 -62.51 ... -61.24 -61.24
    elev              (latitude, longitude) float32 nan nan nan ... nan nan nan
    spatial_ref       int64 0
Data variables:
    reflectance       (latitude, longitude, wavelengths) float32 nan nan ... nan
Attributes: (12/40)
    ncei_template_version:             NCEI_NetCDF_Swath_Template_v2.0
    summary:                           The Earth Surface Mineral Dust Source ...
    keywords:                          Imaging Spectroscopy, minerals, EMIT, ...
    Conventions:                       CF-1.63
    sensor:                            EMIT (Earth Surface Mineral Dust Sourc...
    instrument:                        EMIT
    ...                                ...
    spatial_ref:                       GEOGCS["WGS 84",DATUM["WGS_1984",SPHER...
    geotransform:                      [-6.25120945e+01  5.42232520e-04 -0.00...
    day_night_flag:                    Day
    title:                             EMIT L2A Estimated Surface Reflectance...
    granule_id:                        EMIT_L2A_RFL_001_20220903T163129_22246...
    Orthorectified:                    True

Now, write the dataset as an `.envi` output. If we chose not to orthorectify, you can include a `glt` file to orthorectify later.

In [10]:
et.write_envi(ds, outpath, overwrite=False, extension='.img', interleave='BIL', glt_file=False)